In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import cv2
import os
import tensorflow_datasets as tfds
import datetime
from coco import COCO
from coco_final_labels import labels as coco_labels
import shutil
from tensorflow.keras.utils import to_categorical
import matplotlib as plt
from tensorflow import keras
import random
import tensorflow_addons as tfa
import io


from kerasgen.balanced_image_dataset import balanced_image_dataset_from_directory

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

2022-08-04 01:47:02.201742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 01:47:02.207617: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 01:47:02.207767: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
LAST_DENSE = 1024
SOFTMAX_OUT = True
FROM_LOGITS = False
LEARNING_RATE = 0.0001
INFO = "-"
TRAIN_IMAGES = 'data/coco_onehot_train'
VAL_IMAGES = 'data/coco_onehot_val'
TRANSFER_TRAIN_IMAGES ='data/transfer_train'
TRANSFER_VAL_IMAGES = 'data/transfer_val'
BATCH_SIZE = 16

CONFIG_STRING = "batch_size: {0} | last_dense: {1} | softmax: {2} | from_logits: {3} | lr: {4} | info: {5}"

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

2022-08-04 01:47:04.495767: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-04 01:47:04.496409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 01:47:04.496562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 01:47:04.496674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [4]:
train_files = {}
for d in os.listdir(TRAIN_IMAGES):
    files = os.listdir(f"{TRAIN_IMAGES}/{d}")
    if len(files) > 0:
        train_files[d] = files

print(len(train_files))

val_files = {}
for d in os.listdir(VAL_IMAGES):
    files = os.listdir(f"{VAL_IMAGES}/{d}")
    if len(files) > 0:
        val_files[d] = files

print(len(val_files))

transfer_train_files = {}
for d in os.listdir(TRANSFER_TRAIN_IMAGES):
    files = os.listdir(f"{TRANSFER_TRAIN_IMAGES}/{d}")
    if len(files) > 0:
        transfer_train_files[d] = files

print(len(transfer_train_files))

transfer_val_files = {}
for d in os.listdir(TRANSFER_VAL_IMAGES):
    files = os.listdir(f"{TRANSFER_VAL_IMAGES}/{d}")
    if len(files) > 0:
        transfer_val_files[d] = files

print(len(transfer_val_files))

80
80
6
6


In [5]:
def load_image(image, label):
    # image = tf.io.read_file(filename)
    # image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224,224))
    image = tf.cast(image, tf.float32) / 255.
    return (image, label)

In [6]:
resnet50 = keras.applications.ResNet50V2(include_top=False, weights="imagenet", input_shape=(224,224,3))
resnet50.trainable = True

In [7]:
input = Input(shape=(224,224,3))
encoder = resnet50(input)
avg_pool = GlobalAveragePooling2D()(encoder)
fc = Dense(2048, activation="relu")(avg_pool)
fc = Dense(2048, activation="relu")(fc)
fc = Dense(256, activation=None)(fc)
#output = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(fc)
model = Model(input, fc)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 dense_1 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                             

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.002),
    loss=tfa.losses.TripletSemiHardLoss(),
)

In [13]:
#builder = tfds.ImageFolder(root_dir='data2/')
# print(builder.info)
# ds = builder.as_dataset(split='coco_onehot_train', shuffle_files=True, as_supervised=True)
#val_ds = builder.as_dataset(split='coco_onehot_val', shuffle_files=True, as_supervised=True)

train_ds = balanced_image_dataset_from_directory(
    directory='data2/coco_onehot_train',
    image_size=(224,224),
    validation_split=0,
    subset=None,
    seed=555,
    safe_triplet=True
)

val_ds = balanced_image_dataset_from_directory(
    directory='data2/coco_onehot_val',
    image_size=(224,224),
    validation_split=0,
    subset=None,
    seed=555,
    safe_triplet=True
)

Found 151333 files belonging to 80 classes.
Found 16005 files belonging to 80 classes.


In [10]:
ds = ds.map(load_image)
ds = ds.batch(32, drop_remainder=True)

In [10]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
weight_dir = 'weights/siamese_tripletloss/{0}'.format(current_time)

weight_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=weight_dir,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [11]:
model.fit(train_ds, epochs=20, validation_data=val_ds, callbacks=[weight_callback])

Epoch 1/20


2022-08-04 01:47:43.400977: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-08-04 01:47:43.848502: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-04 01:47:45.106157: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


4730/4730 [==============================] - ETA: 0s - loss: 0.4830

2022-08-04 02:05:00.175568: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: weights/siamese_tripletloss/20220804-014738/assets
4730/4730 [==============================] - 1046s 220ms/step - loss: 0.4830 - val_loss: 1.6237
Epoch 2/20
4730/4730 [==============================] - 1028s 217ms/step - loss: 0.3802 - val_loss: 70.7245
Epoch 3/20
4730/4730 [==============================] - 923s 195ms/step - loss: 0.3664 - val_loss: 0.4748
Epoch 4/20
4730/4730 [==============================] - 906s 192ms/step - loss: 0.3498 - val_loss: 0.7974
Epoch 5/20
4730/4730 [==============================] - 906s 192ms/step - loss: 0.3290 - val_loss: 0.5242
Epoch 6/20
4730/4730 [==============================] - 907s 192ms/step - loss: 0.3172 - val_loss: 0.4823
Epoch 7/20
4730/4730 [==============================] - 907s 192ms/step - loss: 0.3260 - val_loss: 0.6641
Epoch 8/20
4730/4730 [==============================] - 918s 194ms/step - loss: 0.3133 - val_loss: 0.3921
Epoch 9/20
4730/4730 [==============================] - 907s 192ms/step - 

In [12]:
val_ds = val_ds.map(load_image)
val_ds = val_ds.batch(32)



In [14]:
results = model.predict(val_ds)

In [15]:
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open("meta.tsv", "w", encoding="utf-8")
for img, labels in tfds.as_numpy(val_ds):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()

In [16]:
input = next(val_ds.as_numpy_iterator())[0]
print(input)

model.predict(input)

[[[[ 14.         16.         13.       ]
   [ 14.         16.         13.       ]
   [ 14.         16.         13.       ]
   ...
   [ 43.         37.         25.       ]
   [ 43.         37.         25.       ]
   [ 43.         37.         25.       ]]

  [[ 20.         22.         17.       ]
   [ 20.         22.         17.       ]
   [ 20.         22.         17.       ]
   ...
   [ 76.97766    65.01117    52.97766  ]
   [ 75.0134     65.9933     51.013397 ]
   [ 77.         65.         53.       ]]

  [[ 17.         22.         15.       ]
   [ 17.         22.         15.       ]
   [ 17.         22.         15.       ]
   ...
   [ 80.97766    59.01117    44.97766  ]
   [ 79.0134     59.9933     43.013397 ]
   [ 81.         59.         45.       ]]

  ...

  [[120.        107.         75.       ]
   [120.        107.         75.       ]
   [120.        107.         75.       ]
   ...
   [202.        208.        180.       ]
   [202.        208.        180.       ]
   [202.        

array([[ 0.28350025,  0.14702086,  3.747245  , ..., -1.9733976 ,
        -1.2136004 , -0.9483841 ],
       [ 0.47213095,  0.793357  ,  2.4847436 , ..., -1.9187876 ,
        -0.98294044, -0.6913021 ],
       [ 0.8424904 ,  0.4827597 ,  0.9655766 , ..., -2.4185    ,
        -1.324714  , -0.72926736],
       ...,
       [ 0.633972  ,  0.87617934,  0.5870922 , ..., -1.3285888 ,
        -0.49002016, -0.6064803 ],
       [ 0.54578656,  0.07476747,  3.7130547 , ..., -2.0527558 ,
        -1.1251234 , -0.9045205 ],
       [ 0.07311046,  0.12945877,  3.9565244 , ..., -1.4896568 ,
        -1.0014912 , -0.73534906]], dtype=float32)